In [28]:
import sqlite3
import glob
import urllib.parse
import records
import requests
from bs4 import BeautifulSoup
import MeCab as mc
import re
from collections import Counter
from matplotlib import pyplot as plt
import gensim
from gensim.models import Word2Vec
from gensim import matutils, corpora
from gensim.corpora.dictionary import Dictionary
from joblib import Parallel

In [11]:
conn = None

# データベースに接続する
def connect():
    global conn
    conn = sqlite3.connect("./Restaurants_Reviews_unique.db")
    
# コネクションを断つ
def close():
    conn.close()
    
# テーブル作成
def create_table_Restaurants():
    # DROP=消す.
    conn.execute("DROP TABLE IF EXISTS Restaurants")
    conn.execute("""CREATE TABLE Restaurants (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            rest_name TEXT
    )""")
    
def create_table_Reviews():
    # DROP=消す.
    conn.execute("DROP TABLE IF EXISTS Reviews")
    conn.execute("""CREATE TABLE Reviews (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            rest_id INT,
            review TEXT,
            rate FLOAT
    )""")
    
# loadは積むという意味
def load_Restaurants(rest_name):
    conn.execute("INSERT INTO Restaurants (rest_name) VALUES (?)", (rest_name,))
    conn.commit()
    
def load_Reviews(rest_id, review, rate):
    conn.execute(
        "INSERT INTO Reviews (rest_id,review,rate) VALUES (?,?,?)", (rest_id,review,rate))
    conn.commit()

In [3]:
url = 'https://tabelog.com/rstLst/hamburger/?SrtT=rvcn&sk=ハンバーガー&svd=20190530&svt=1900&svps=2&Srt=D'

r = requests.get(url)
html_soup = BeautifulSoup(r.text, 'html.parser')
# 一番初めのurl

In [4]:
new_url_all = [r["href"] for r in html_soup.find_all("a", class_ ="list-rst__rvw-count-target")]
print(new_url_all)
# 20件のurl

rest_name_all = [r.text for r in html_soup.find_all("a", class_ ="list-rst__rst-name-target cpy-rst-name")]
print(rest_name_all)

try:
    connect()
    create_table_Restaurants()
    for rest_name in rest_name_all:
        load_Restaurants(rest_name)
finally:
    close()

['https://tabelog.com/tokyo/A1302/A130204/13003002/dtlrvwlst/', 'https://tabelog.com/tokyo/A1310/A131004/13003538/dtlrvwlst/', 'https://tabelog.com/tokyo/A1314/A131404/13004775/dtlrvwlst/', 'https://tabelog.com/tokyo/A1303/A130302/13096437/dtlrvwlst/', 'https://tabelog.com/tokyo/A1306/A130601/13042654/dtlrvwlst/', 'https://tabelog.com/tokyo/A1316/A131603/13001556/dtlrvwlst/', 'https://tabelog.com/tokyo/A1314/A131401/13121856/dtlrvwlst/', 'https://tabelog.com/tokyo/A1317/A131706/13004780/dtlrvwlst/', 'https://tabelog.com/tokyo/A1304/A130401/13045308/dtlrvwlst/', 'https://tabelog.com/tokyo/A1303/A130301/13054359/dtlrvwlst/', 'https://tabelog.com/tokyo/A1317/A131707/13022762/dtlrvwlst/', 'https://tabelog.com/hokkaido/A0105/A010501/1000019/dtlrvwlst/', 'https://tabelog.com/tokyo/A1324/A132402/13033094/dtlrvwlst/', 'https://tabelog.com/tokyo/A1306/A130603/13189009/dtlrvwlst/', 'https://tabelog.com/tokyo/A1310/A131003/13006525/dtlrvwlst/', 'https://tabelog.com/okinawa/A4701/A470103/47000177/

In [10]:
review_all = []
try:
    connect()
    create_table_Reviews()
    for i,new_url in enumerate(new_url_all):
        burger_r = requests.get(new_url)
        burger_soup = BeautifulSoup(burger_r.text, 'html.parser')
        # 件数クリックしたurl
        
        review_all = [r["data-detail-url"] for r in burger_soup.find_all("div", class_ ="rvw-item js-rvw-item-clickable-area")]

        #         print(review_all)
        for j in range(int(int(burger_soup.find("em", class_="num").text)/20)+1):
            for review in review_all:
    #             print(i+1)
                review_r = requests.get("https://tabelog.com"+review+"COND-0/smp1/?smp=1&lc=0&rvw_part=all&PG="+str(j+1))
                review_soup = BeautifulSoup(review_r.text, 'html.parser')

                rest_review = review_soup.find("div", class_ ="rvw-item__rvw-comment").text
#                 print(rest_review)
                if review_soup.find("b", class_ ="c-rating__val c-rating__val--strong").text == "-":
                    continue
                rate = float(review_soup.find("b", class_ ="c-rating__val c-rating__val--strong").text)
#                 print(rate)
#                 load_Reviews(i+1, rest_review, rate)
finally:
    close()

KeyboardInterrupt: 

In [30]:
try:
    connect()
    create_table_Reviews()
    for i,new_url in enumerate(new_url_all):
        burger_r = requests.get(new_url)
        burger_soup = BeautifulSoup(burger_r.text, 'html.parser')
        # 件数クリックしたurl
        
        review_all = []
        for j in range(int(int(burger_soup.find("em", class_="num").text)/20)+1): # 1ページづつのレビュー
            review_r = requests.get(new_url+"COND-0/smp1/?smp=1&lc=0&rvw_part=all&PG="+str(j+1))
            review_soup = BeautifulSoup(review_r.text, 'html.parser')
            review_all += [r["data-detail-url"] for r in review_soup.find_all("div", class_ ="rvw-item js-rvw-item-clickable-area")]
            
        for review in review_all:
#             print(i+1)
            review_content = requests.get("https://tabelog.com"+review)
            review_content_soup = BeautifulSoup(review_content.text, 'html.parser')

            rest_review = review_content_soup.find("div", class_ ="rvw-item__rvw-comment").text
#             print(rest_review)
            if review_content_soup.find("b", class_ ="c-rating__val c-rating__val--strong").text == "-":
                continue
            rate = float(review_content_soup.find("b", class_ ="c-rating__val c-rating__val--strong").text)
#             print(rate)
            load_Reviews(i+1, rest_review, rate)
finally:
    close()

ConnectionError: HTTPSConnectionPool(host='tabelog.com', port=443): Max retries exceeded with url: /tokyo/A1302/A130204/13003002/dtlrvwlst/B111369379/?use_type=0&srt=&sby=&rvw_part=all&lc=0&smp=1 (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x7effa5c1cc88>: Failed to establish a new connection: [Errno 11] Resource temporarily unavailable',))

In [26]:
new_url_all

['https://tabelog.com/tokyo/A1302/A130204/13003002/dtlrvwlst/',
 'https://tabelog.com/tokyo/A1310/A131004/13003538/dtlrvwlst/',
 'https://tabelog.com/tokyo/A1314/A131404/13004775/dtlrvwlst/',
 'https://tabelog.com/tokyo/A1303/A130302/13096437/dtlrvwlst/',
 'https://tabelog.com/tokyo/A1306/A130601/13042654/dtlrvwlst/',
 'https://tabelog.com/tokyo/A1316/A131603/13001556/dtlrvwlst/',
 'https://tabelog.com/tokyo/A1314/A131401/13121856/dtlrvwlst/',
 'https://tabelog.com/tokyo/A1317/A131706/13004780/dtlrvwlst/',
 'https://tabelog.com/tokyo/A1304/A130401/13045308/dtlrvwlst/',
 'https://tabelog.com/tokyo/A1303/A130301/13054359/dtlrvwlst/',
 'https://tabelog.com/tokyo/A1317/A131707/13022762/dtlrvwlst/',
 'https://tabelog.com/hokkaido/A0105/A010501/1000019/dtlrvwlst/',
 'https://tabelog.com/tokyo/A1324/A132402/13033094/dtlrvwlst/',
 'https://tabelog.com/tokyo/A1306/A130603/13189009/dtlrvwlst/',
 'https://tabelog.com/tokyo/A1310/A131003/13006525/dtlrvwlst/',
 'https://tabelog.com/okinawa/A4701/A4